In [1]:
import sys
sys.path.append("..")
import os
import numpy as np
import tensorflow as tf
from gan.network import DoppelGANgerGenerator, Discriminator, RNNInitialStateType, AttrDiscriminator
from gan.doppelganger import DoppelGANger
from gan.output import OutputType, Normalization, Output
from gan.util import add_gen_flag, normalize_per_sample

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from matplotlib import pyplot as plt

c:\Users\jankr\miniconda3\envs\tf2\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.7.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [2]:
data_feature_outputs = [
    Output(type_=OutputType.CONTINUOUS, dim=1, normalization=Normalization.MINUSONE_ONE, is_gen_flag=False)] # Consumption
data_attribute_outputs = [
    Output(type_=OutputType.CONTINUOUS, dim=1, normalization=Normalization.ZERO_ONE, is_gen_flag=False), #Yearly Offtake
    Output(type_=OutputType.CONTINUOUS, dim=1, normalization=Normalization.ZERO_ONE, is_gen_flag=False), #Yearly Injection
    Output(type_=OutputType.CONTINUOUS, dim=1, normalization=Normalization.ZERO_ONE, is_gen_flag=False), # Feels Like C
    Output(type_=OutputType.CONTINUOUS, dim=1, normalization=Normalization.ZERO_ONE, is_gen_flag=False), # sunHour
    Output(type_=OutputType.DISCRETE, dim=4, normalization=None, is_gen_flag=False), #season
    Output(type_=OutputType.DISCRETE, dim=2, normalization=None, is_gen_flag=False)] #is_weekend

Load data

In [3]:
data = np.load('../../data/train/train.npz', allow_pickle=True)
data_feature = np.expand_dims(data['f'],axis=2)
data_attribute = data['a'].astype(float)
data_gen_flag = np.ones((data_feature.shape[0],data_feature.shape[1]))

In [4]:
max_value = np.max(data_feature)
min_value = np.min(data_feature)

# Scale the array to be between -1 and 1
normalized_array = (data_feature - min_value) * (2 / (max_value - min_value)) - 1

apply auto-normalization

In [5]:
sample_len = 96 #run 1, 12, run 11 -> 12

# (data_feature, data_attribute, \
#         data_attribute_outputs, real_attribute_mask) = normalize_per_sample(data_feature, 
#                                                                                 data_attribute, 
#                                                                                 data_feature_outputs,
#                                                                                 data_attribute_outputs)

real_attribute_mask = [True] * len(data_attribute_outputs)
data_feature, data_feature_outputs = add_gen_flag(data_feature,
                                                        data_gen_flag,
                                                        data_feature_outputs,
                                                        sample_len)

In [6]:
initial_state = RNNInitialStateType.RANDOM #run 10 changed from RANDOM to variable, run 11, changed back

generator = DoppelGANgerGenerator(
    feed_back=False,
    noise=True,
    feature_outputs=data_feature_outputs,
    attribute_outputs=data_attribute_outputs,
    real_attribute_mask=real_attribute_mask,
    sample_len=sample_len,
    feature_num_layers=2,       #run 1: 1, run5:5, run 14 -> (1, 100 layers, 1, 100 layers)
    feature_num_units=100,      #run before 5, 100
    attribute_num_layers=2,     #run 8: -> 2
    attribute_num_units=100,    #run before 5, 100, run 8:100
    initial_state=initial_state)

discriminator = Discriminator(
    num_layers=5,
    num_units=200)

attr_discriminator = AttrDiscriminator(
    num_layers=5,
    num_units=200)

In [7]:
run = 'run17_withmetrics'

checkpoint_dir = os.path.join('results',run, "checkpoint")
os.makedirs(checkpoint_dir,exist_ok=True)
sample_dir = os.path.join('results',run, "sample")
os.makedirs(sample_dir,exist_ok=True)
time_path = os.path.join('results',run, "time.txt")

In [8]:
# epoch: Number of training epochs.
# batch_size: Training batch size.
# sample_len: The time series batch size, i.e. the number of time steps generated at each RNN rollout (parameter S in Section 4.1).
# aux_disc: Whether to have auxiliary discriminator (Section 4.3).
# self_norm: Whether to do auto-normalization for each time series (Section 4.2).
# num_packing: Packing degree in PacGAN (a method for solving mode collapse in NeurIPS 2018, see the paper and code). Setting it to 1 is equivalent to the vanilla GAN without packing.
# noise: Whether to input noise at each RNN rollout.
# feed_back: Whether to input the generated values from the previous rollout at each RNN rollout.
# g_lr: The learning rate in Adam for training the generator
# d_lr: The learning rate in Adam for training the discriminator.
# d_gp_coe: Weight of gradient penalty loss in Wasserstein GAN for the discriminator.
# attr_d_lr: The learning rate in Adam for training the auxiliary discriminator.
# attr_d_gp_coe: Weight of gradient penalty loss in Wasserstein GAN for the auxiliary discriminator.
# d_rounds: Number of discriminator steps per batch.
# g_rounds: Number of generator steps per batch.
# gen_feature_num_layers: Number of layers in the time series generator (RNN).
# gen_feature_num_units: Number of units in each layer of the time series generator (RNN).
# gen_attribute_num_layers: Number of layers in the attribute (metadata) generator.
# gen_attribute_num_units: Number of units in each layer of the attribute (metadata) generator.
# attr_disc_num_layers: Number of layers in the auxiliary discriminator.
# attr_disc_num_units: Number of units in each layer of the auxiliary discriminator.
# disc_num_layers: Number of layers in the discriminator.
# disc_num_units: Number of units in each layer of the auxiliary discriminator.
# initial_state: The initial state for the RNN: "random" means setting the initial state to random numbers; "zero" means setting the initial state to zeros; "variable" means setting the inital state to be learnable parameters.
# extra_checkpoint_freq: The frequency of saving the trained model in a separated folder (unit: epoch).
# epoch_checkpoint_freq: The frequency of saving the trained model (unit: epoch).
# vis_freq: The frequency of visualizing generated samples during training (unit: training batch).
# vis_num_sample: The number of samples to visualize each time during training.

In [9]:
run_config = tf.compat.v1.ConfigProto()
with tf.compat.v1.Session(config=run_config) as sess:
        gan = DoppelGANger(
    sess=sess,
    checkpoint_dir=checkpoint_dir,
    sample_dir=sample_dir,
    time_path=time_path,
    epoch=1000,
    batch_size=100,
    data_feature=data_feature,
    data_attribute=data_attribute,
    real_attribute_mask=real_attribute_mask,
    data_gen_flag=data_gen_flag,
    sample_len=sample_len,
    data_feature_outputs=data_feature_outputs,
    data_attribute_outputs=data_attribute_outputs,
    vis_freq=200,
    vis_num_sample=5,
    generator=generator,
    discriminator=discriminator,
    attr_discriminator=attr_discriminator,
    d_gp_coe=10.0,
    attr_d_gp_coe=10.0,
    g_attr_d_coe=1.0,
    d_rounds=1,
    g_rounds=1,
    g_lr=0.001,
    d_lr=0.001,
    attr_d_lr=0.001,
    extra_checkpoint_freq=5,
    epoch_checkpoint_freq=5,
    num_packing=1)
        gan.build()
        gan.train(restore=False)

Tensor("g_feature_input_data_train_0:0", shape=(None, 288), dtype=float32)
(None, 288)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


c:\Thesis\DoppelGANger-TF2\scripts_generate\..\gan\op.py:11: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  output = tf.compat.v1.layers.dense(
c:\Users\jankr\miniconda3\envs\tf2\lib\site-packages\keras\legacy_tf_layers\core.py:255: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
c:\Thesis\DoppelGANger-TF2\scripts_generate\..\gan\network.py:299: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(
c:\Users\jankr\miniconda3\envs\tf2\lib\site-packages\keras\layers\legacy_rnn\rnn_cell_impl.py:984: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` 

Tensor("g_feature_input_data_test_teacher:0", shape=(None, None, 288), dtype=float32)
(None, None, 288)
Tensor("g_feature_input_data_test_free:0", shape=(None, 288), dtype=float32)
(None, 288)
Tensor("g_feature_input_data_test_free:0", shape=(None, 288), dtype=float32)
(None, 288)
Layers of DoppelGANgerGenerator
[<tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/linear/dense/kernel:0' shape=(5, 100) dtype=float32>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/linear/dense/bias:0' shape=(100,) dtype=float32>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/batch_norm_1/gamma:0' shape=(100,) dtype=float32>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/batch_norm_1/beta:0' shape=(100,) dtype=float32>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/batch_norm_1/moving_mean:0' shape=(100,) dtype=float32>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/batch_norm_1/moving_variance:0' shape=(100,) dtype=float32>, <tf.Variable '

  3%|▎         | 29/1000 [12:41<8:32:45, 31.68s/it]

Instructions for updating:
Use standard file APIs to delete files with this prefix.


 50%|█████     | 501/1000 [8:18:10<8:16:11, 59.66s/it]  
